In [1]:
!nvidia-smi

Mon Mar 22 01:15:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   35C    P0    38W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# setup

In [2]:
%load_ext autoreload
%autoreload 2
%aimport

HOME = '/data/git/shopee-product-matching'
import sys
sys.path.append(f"{HOME}/src")

p_in = f'{HOME}/input'
p_comp = f'{p_in}/shopee-product-matching'
p_out = f'{HOME}/output/dev0017'

!mkdir -p {p_out}
from utils import *
import tmdl
import vmdl
from vmdl import VDataset, tfms



Modules to reload:
all-except-skipped

Modules to skip:



In [3]:
p_imgs = f"{p_comp}/test_images"
df = pd.read_csv(f'{p_comp}/test.csv')
df.shape

(3, 4)

In [4]:
p_imgs = f"{p_comp}/train_images"
df = pd.read_csv(f'{p_comp}/train.csv', nrows=10000)
df.shape

(10000, 5)

In [5]:
pids = df.posting_id.values

In [11]:
from efficientnet_pytorch import EfficientNet
mdl = EfficientNet.from_pretrained("efficientnet-b0")

feats_v = vmdl.mk_feats(df, p_imgs, mdl)

Loaded pretrained weights for efficientnet-b0


In [12]:
from sklearn.neighbors import NearestNeighbors

In [13]:
%%time
nbrs = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(feats_v)
distances, indices = nbrs.kneighbors(feats_v)

CPU times: user 6.18 s, sys: 847 ms, total: 7.02 s
Wall time: 7.02 s


In [14]:
distances

array([[ 0.        , 11.07120937, 11.13309529, ..., 11.59859328,
        11.61007676, 11.61427486],
       [ 0.        ,  5.73584561,  5.94093856, ..., 11.43051128,
        11.43128658, 11.57397681],
       [ 0.        , 12.29418141, 12.37679559, ..., 13.27385638,
        13.30235227, 13.31632692],
       ...,
       [ 0.        , 11.78373288, 12.01131892, ..., 12.70817801,
        12.7285922 , 12.75987869],
       [ 0.        ,  5.32917788, 12.31732296, ..., 13.39661869,
        13.41258765, 13.44934583],
       [ 0.        , 12.3615117 , 12.44734191, ..., 13.01533828,
        13.26739633, 13.36687971]])

In [15]:
indices

array([[   0,  686, 9813, ..., 4189, 8372, 2135],
       [   1,  469,  194, ..., 3126, 9441, 5680],
       [   2, 3446, 7973, ..., 4316, 4094,  435],
       ...,
       [9997, 7157, 5996, ..., 9066, 7122, 9024],
       [9998, 6770, 1866, ..., 8352, 1752,  260],
       [9999, 8777, 1990, ..., 4967, 1217, 6403]])